# Testing how compensations happen

In [2]:
include("../BK_functions/bk_setup_script.jl"); 

In [6]:
# Load the parameters from Fig 2 example:
using JLD
phi1_orig = load("results/noisy_fits_8.jld", "noisy_fits")[10]["phi1_orig"]
#phi1_orig = load("results/noisy_fits_5.jld", "noisy_fits")[10]["phi1_orig"]

8-element Array{Float64,1}:
 7.69353e-37
 8.75658e-33
 0.0662813  
 0.586839   
 2.74314e-5 
 1.72186e8  
 2.39839e9  
 4.54419e-8 

In [7]:
# Simulate log data from this fit
#ydata_noiseless = BK_simulator(phi1_orig, hcat(x_grid...), model_id=0);
ydata_noiseless = BK_simulator(phi0, hcat(x_grid...), model_id=0); # try phi0 again?
ydata_log = log10(ydata_noiseless).*(1+(2*rand(size(ydata_noiseless))-1)*0.1) # Add up to 10% noise (uniform, multiplicative, but in log space assay)

LoadError: LoadError: UndefVarError: BK_simulator not defined
while loading In[7], in expression starting on line 2

In [48]:
outp_log10 = load("results/MBAM_outp_log10.jld", "outp_log10");

# Choose models and corresponding phi fit
model_log_id = 51; 

phi0_log = exp(outp_log10[1][1,1])

using LsqFit
# Store multiple fits (different initial point for LM, takes around 5 sec / fit)
all_fits = Array(Any, 24)
all_costs = Array(Any, length(all_fits))

for i1 = 1:length(all_fits)
    if i1==1
        phi_init = phi0_log;
    else
        phi_init = phi0_log.*(1+(2*rand(size(phi0_log))-1)*0.9) # Add up to 90% noise to initial params
    end
    all_fits[i1] = curve_fit((x,p) -> BK_simulator(exp(p), x, model_id=model_log_id), hcat(x_grid...), ydata_log, log(phi_init)); # abs needed due to weird julia error, BK_simulator has an output range between 0 and 1
    all_costs[i1] = sum(all_fits[i1].resid.^2);
end

# Find the best fit
phi1_log = exp(all_fits[findmin(all_costs)[2]].param);

# Saving fit results:
res_51 = Dict()
res_51["ydata_log"] = ydata_log
res_51["phi0_log"] = phi0_log
res_51["phi1_log"] = phi1_log

In [ ]:
# Choose models and corresponding phi fit
model_log_id = 52; 

#phi0_log = phi1_orig[[1;3:8]]# exp(outp_log10[1][2,1])

phi0_log_high = load("results/noisy_fits_to_diverging_fit8_trueinit_"*string(5)*".jld", "noisy_fits")[52]["phi1_log"]
phi0_log_low = load("results/noisy_fits_to_diverging_fit8_trueinit_"*string(1)*".jld", "noisy_fits")[52]["phi1_log"]


using LsqFit
# Store multiple fits (different initial point for LM, takes around 5 sec / fit)
all_fits = Array(Any, 24)
all_costs = Array(Any, length(all_fits))

for i1 = 1:length(all_fits)
    println(i1)
    if i1<=12
        phi_init = phi0_log_low.*(1+(2*rand(size(phi0_log_low))-1)*0.9) # Add up to 90% noise to initial params
    else
        phi_init = phi0_log_high.*(1+(2*rand(size(phi0_log_high))-1)*0.9) # Add up to 90% noise to initial params
    end
    all_fits[i1] = curve_fit((x,p) -> BK_simulator(exp(p), x, model_id=model_log_id), hcat(x_grid...), ydata_log, log(phi_init)); # abs needed due to weird julia error, BK_simulator has an output range between 0 and 1
    all_costs[i1] = sum(all_fits[i1].resid.^2);
end

# Find the best fit
phi1_log = exp(all_fits[findmin(all_costs)[2]].param);

# Saving fit results:
res_52 = Dict()
res_52["ydata_log"] = ydata_log
res_52["phi0_log"] = phi0_log
res_52["phi1_log"] = phi1_log

# Choose models and corresponding phi fit
model_log_id = 53; 

phi0_log = exp(outp_log10[1][3,1])

using LsqFit
# Store multiple fits (different initial point for LM, takes around 5 sec / fit)
all_fits = Array(Any, 24)
all_costs = Array(Any, length(all_fits))

for i1 = 1:length(all_fits)
    println(i1)
    if i1==1
        phi_init = phi0_log;
    else
        phi_init = phi0_log.*(1+(2*rand(size(phi0_log))-1)*0.9) # Add up to 90% noise to initial params
    end
    all_fits[i1] = curve_fit((x,p) -> BK_simulator(exp(p), x, model_id=model_log_id), hcat(x_grid...), ydata_log, log(phi_init)); # abs needed due to weird julia error, BK_simulator has an output range between 0 and 1
    all_costs[i1] = sum(all_fits[i1].resid.^2);
end

# Find the best fit
phi1_log = exp(all_fits[findmin(all_costs)[2]].param);

# Saving fit results:
res_53 = Dict()
res_53["ydata_log"] = ydata_log
res_53["phi0_log"] = phi0_log
res_53["phi1_log"] = phi1_log

metadata = Dict("data_noise"=>"10%", "num_inits"=>24, "init_param_noise"=>"90%", "phi1_orig"=>phi1_orig)

noisy_fits = Dict(51=>res_51, 52=>res_52, 53=>res_53)

In [ ]:
metadata = Dict("data_noise"=>"10%", "num_inits"=>24, "init_param_noise"=>"90%", "phi1_orig"=>phi1_orig)

noisy_fits = Dict(52=>res_52)

In [ ]:
#save results - 10% version (original)
save("results/noisy_fits_globalinit_"*string(CURDATAPOINT)*".jld", "noisy_fits", noisy_fits, "metadata", metadata);